In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# dataset = load_dataset('cos_e', 'v1.11')

In [2]:
from datasets import load_from_disk


# Example to load from the directory named "formatted_dataset"
loaded_dataset = load_from_disk('./formatted_dataset')

# Verify the loaded dataset
print(loaded_dataset['train'][1]['generated_output'])
print(loaded_dataset['validation'][1]['generated_output'])


$answer$ = It's wrong to make generalizations about people in certain situations.
$answer$ = It's good to give someone a ride who needs one.


remove \n$answer$

In [5]:
def remove_answer_prefix(example):
    if 'generated_output' in example:
        example['generated_output'] = [output.replace("$answer$ =", "").strip() for output in example['generated_output']]
    return example


In [6]:
from datasets import load_from_disk

# Apply the function to the dataset
transformed_dataset = loaded_dataset.map(remove_answer_prefix, batched=True)

# Verify the transformation
print(transformed_dataset['train'][0]['generated_output'])
print(transformed_dataset['validation'][0]['generated_output'])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

It is good to be aware of your surroundings.



In [11]:
print([transformed_dataset['validation'][0]['generated_output']])

['']


In [7]:
# Verify the transformation
print(transformed_dataset['train'][1]['generated_output'])
print(transformed_dataset['validation'][1]['generated_output'])

It's wrong to make generalizations about people in certain situations.
It's good to give someone a ride who needs one.


In [20]:
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'formatted_question', 'generated_output'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'formatted_question', 'generated_output'],
        num_rows: 1221
    })
})

In [13]:
# Initialize the model and tokenizer
model_name = "t5-base"  # Use "t5-base" or "t5-large" if resources allow
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

must be change

In [ ]:
# # Preprocess the dataset
# def preprocess_function(examples):
#     inputs = ["question: " + q + " answer: " + " ".join(choices) for q, choices in zip(examples['question'], examples['choices'])]
#     targets = ["answer: " + answer + " explanation: " + explanation for answer, explanation in zip(examples['answer'], examples['abstractive_explanation'])]
#     model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length')
#     labels = tokenizer(targets, max_length=256, truncation=True, padding='max_length')
#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

In [21]:
# Preprocess the dataset
def preprocess_function(examples):
    # Check if 'generated_output' exists in the examples
    if 'generated_output' not in examples:
        raise ValueError("The 'generated_output' field is missing in the dataset examples.")

    # Concatenate each generated_output to the end of the input
    inputs = ["question: " + q + " answer: " + " ".join(choices) + " " + generated_output
              for q, choices, generated_output in zip(examples['question'], examples['choices'], examples['generated_output'])]
    
    targets = ["answer: " + answer + " explanation: " + explanation 
               for answer, explanation in zip(examples['answer'], examples['abstractive_explanation'])]
    
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=256, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

# Example usage






In [22]:
encoded_dataset = transformed_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [24]:
# Print a few examples to verify the preprocessing
print(encoded_dataset['train'][0]['input_ids'])
print(encoded_dataset['train'][0]['labels'])
print(encoded_dataset['validation'][0]['input_ids'])
print(encoded_dataset['validation'][0]['labels'])

[822, 10, 96, 7238, 33, 335, 16981, 30, 46, 8947, 2195, 5, 5245, 1590, 326, 5, 852, 132, 33, 3, 4, 16981, 535, 363, 19, 48, 46, 677, 13, 58, 1525, 10, 2447, 7089, 484, 2004, 1530, 7270, 682, 18076, 94, 19, 207, 12, 36, 2718, 13, 39, 14338, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1525, 10, 7270, 682, 7295, 10, 765, 3357, 107, 19, 876, 12, 199, 25, 4602, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [25]:
# Function to decode the encoded inputs and labels for validation
def decode_example(example):
    input_ids = example['input_ids']
    labels = example['labels']
    
    decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
    decoded_label = tokenizer.decode(labels, skip_special_tokens=True)
    
    return decoded_input, decoded_label

# Decode and print a few examples from the training set
for i in range(3):
    decoded_input, decoded_label = decode_example(encoded_dataset['train'][i])
    print(f"Example {i + 1} - Decoded Input: {decoded_input}")
    print(f"Example {i + 1} - Decoded Label: {decoded_label}\n")

# Decode and print a few examples from the validation set
for i in range(3):
    decoded_input, decoded_label = decode_example(encoded_dataset['validation'][i])
    print(f"Example {i + 1} - Decoded Input: {decoded_input}")
    print(f"Example {i + 1} - Decoded Label: {decoded_label}\n")



Example 1 - Decoded Input: question: "There are 10 apples on an apple tree. Three fall off. Now there are X apples." What is this an example of? answer: park coloring book garden center math problem gravity It is good to be aware of your surroundings.
Example 1 - Decoded Label: answer: math problem explanation: webmath is designed to help you solve

Example 2 - Decoded Input: question: A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live? answer: bus depot beach train station bridge bridge It's wrong to make generalizations about people in certain situations.
Example 2 - Decoded Label: answer: bridge explanation: bums are well known to take up residence under bridges.

Example 3 - Decoded Input: question: A bad person places little value on being honest, acting without pretense or being what? answer: excellent upright premium competent sincere It's wrong to be a bad person.
Example 3 - Decoded Label: answer: sincere exp

In [27]:
encoded_dataset['validation']

Dataset({
    features: ['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'formatted_question', 'generated_output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1221
})

In [28]:
from torch.utils.data import DataLoader

# Convert dataset to PyTorch tensors
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create DataLoaders
train_loader = DataLoader(encoded_dataset['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(encoded_dataset['validation'], batch_size=16)

In [29]:
# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/home/manavi/myenv/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Training loop
def train_loop(model, loader, optimizer, accumulation_steps=2):
    model.train()
    batch_losses = []
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc='Training:')):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss / accumulation_steps  # normalize loss

        batch_loss_value = loss.item() * accumulation_steps  # convert to original loss value for logging
        loss.backward()

        if (i + 1) % accumulation_steps == 0:  # update weights every accumulation_steps mini-batches
            optimizer.step()
            optimizer.zero_grad()  # reset gradients

        batch_losses.append(batch_loss_value)

    # Update remaining gradients if the number of batches is not a multiple of accumulation_steps
    if len(loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    loss_value = sum(batch_losses) / len(batch_losses)
    return {'train_loss': loss_value}

In [32]:
def convert_to_sentences(list_of_lists):
    sentences = [' '.join(inner_list) for inner_list in list_of_lists]
    return sentences

In [33]:
import bert_score
from datasets import load_metric
def validate_loop(model, loader):
    model.eval()
    batch_losses = []
    accuracy_preds = []
    accuracy_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validation:'):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
            loss = outputs.loss

            # Generate predictions
            predictions = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)

            # Decode predictions and labels
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Print decoded predictions and labels for debugging
            # print("Decoded predictions:")
            # for pred in decoded_preds:
            #     print(f"'{pred}'")
            # print("Decoded labels:")
            # for label in decoded_labels:
            #     print(f"'{label}'")

            # # Extract the answers from decoded predictions and labels
            # extracted_preds = [pred.strip().split('.')[0] for pred in decoded_preds if pred.strip()]
            # extracted_labels = [label.strip().split('.')[0] for label in decoded_labels if label.strip()]
            # Extract the answers from decoded predictions and labels
            extracted_preds = [pred.split('answer: ')[1].split(' ')[0] for pred in decoded_preds if 'answer: ' in pred]
            extracted_labels = [label.split('answer: ')[1].split(' ')[0] for label in decoded_labels if 'answer: ' in label]
            
            explanations_preds = [pred.split('explanation: ')[1].split(' ') for pred in decoded_preds if 'explanation: ' in pred]
            explanations_labels = [label.split('explanation: ')[1].split(' ') for label in decoded_labels if 'explanation: ' in label]
            # print(convert_to_sentences(explanations_preds))
            # print(convert_to_sentences(explanations_labels))
            
            # Ensure lengths match for accuracy calculation
            if len(extracted_preds) == len(extracted_labels):
                accuracy_preds.extend(extracted_preds)
                accuracy_labels.extend(extracted_labels)

            batch_losses.append(loss.item())

    # Calculate accuracy
    correct = sum(p == l for p, l in zip(accuracy_preds, accuracy_labels))
    accuracy = correct / len(accuracy_preds) if accuracy_preds else 0.0
    
    
    # Calculate BERTScore for explanations (explanations_preds and explanations_labels)
    P_exp, R_exp, F1_exp = bert_score.score(convert_to_sentences(explanations_preds), convert_to_sentences(explanations_labels), lang="en", rescale_with_baseline=True)
    bertscore_exp_avg = F1_exp.mean().item()

    loss_value = sum(batch_losses) / len(batch_losses)
    # return {'val_loss': loss_value, 'accuracy': accuracy}
    # return {'val_loss': loss_value, 'bertscore_exp': bertscore_exp_avg}
    return {'val_loss': loss_value, 'accuracy': accuracy, 'bertscore_exp': bertscore_exp_avg}


# # Training and validation
# num_epochs = 1
# for epoch in range(num_epochs):
#     train_metrics = train_loop(model, train_loader, optimizer)
#     val_metrics = validate_loop(model, val_loader)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_metrics['train_loss']:.4f}")
#     print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
#     print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    
# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_metrics = train_loop(model, train_loader, optimizer)
    val_metrics = validate_loop(model, val_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_metrics['train_loss']:.4f}")
    print(f"Validation Loss: {val_metrics['val_loss']:.4f}")
    print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"Validation BERTScore (Explanations): {val_metrics['bertscore_exp']:.4f}")



Validation:: 100%|██████████| 77/77 [01:15<00:00,  1.03it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Train Loss: 0.1387
Validation Loss: 0.1272
Validation Accuracy: 0.3104
Validation BERTScore (Explanations): 0.0741


Validation:: 100%|██████████| 77/77 [01:16<00:00,  1.01it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 2/3
Train Loss: 0.1257
Validation Loss: 0.1228
Validation Accuracy: 0.3636
Validation BERTScore (Explanations): 0.0827


Validation:: 100%|██████████| 77/77 [01:16<00:00,  1.01it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 3/3
Train Loss: 0.1197
Validation Loss: 0.1205
Validation Accuracy: 0.4046
Validation BERTScore (Explanations): 0.0300
